# Combine data
The purpose of this notebook is to merge the ORES ranking data, regional data, and population data together. All files have fields containing state names which is used as a primary key to merge them. The output is a single CSV file called: `wp_scored_city_articles_by_state.csv`


In [15]:
import pandas as pd

### Step 1: Joining ORES scoring and regional data

The first step we do is load in the scoring and regional datasets, the ORES scores stored in `us_cities_score_sorted.csv` and the regions stored in `US States by Region - US Census Bureau.xlsx`. 

In [16]:
us_cities_scored = pd.read_csv("../data_intermediate/us_cities_score_sorted.csv")
us_regions = pd.read_excel("../data_raw/US States by Region - US Census Bureau.xlsx")

# Fixing column format
us_cities_scored['revision_id'] = us_cities_scored['revision_id'].astype('string')

Looking at the two dataframes we can see that we need to do a bit of work to merge them together. This is because the `us_regions` dataframe is formatted in a strange manner that contains a lot of NaNs

In [17]:
us_cities_scored.head()

,state,page_title,revision_id,score,url
0,Alabama,"Abbeville, Alabama",1171163550,C,"https://en.wikipedia.org/wiki/Abbeville,_Alabama"
1,Alabama,"Abbeville, Alabama",1171163550,C,"https://en.wikipedia.org/wiki/Abbeville,_Alabama"
2,Alabama,"Adamsville, Alabama",1177621427,C,"https://en.wikipedia.org/wiki/Adamsville,_Alabama"
3,Alabama,"Adamsville, Alabama",1177621427,C,"https://en.wikipedia.org/wiki/Adamsville,_Alabama"
4,Alabama,"Addison, Alabama",1168359898,C,"https://en.wikipedia.org/wiki/Addison,_Alabama"


In [5]:
us_regions.head()

,REGION,DIVISION,STATE
0,Northeast,NaN,NaN
1,NaN,New England,NaN
2,NaN,NaN,Connecticut
3,NaN,NaN,Maine
4,NaN,NaN,Massachusetts


Below, I will reformat the us_regions dataframe so that each state is its own individual row and there is no sparsity.

In [6]:
state_region_dict = {
    "state": [],
    "region": [],
    "division": []
}
current_region = None
current_division = None
for row in us_regions.itertuples():
    if not isinstance(row.REGION, float):
        current_region = row.REGION
    if not isinstance(row.DIVISION, float):
        current_division = row.DIVISION
    if not isinstance(row.STATE, float):
        state_region_dict['state'].append(row.STATE)
        state_region_dict['region'].append(current_region)
        state_region_dict['division'].append(current_division)

us_regions_reformat = pd.DataFrame(state_region_dict)

Below we can see the reformatted regional information.

In [7]:
us_regions_reformat.head()

,state,region,division
0,Connecticut,Northeast,New England
1,Maine,Northeast,New England
2,Massachusetts,Northeast,New England
3,New Hampshire,Northeast,New England
4,Rhode Island,Northeast,New England


In [18]:
us_cities_scored['state'].unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Delaware', 'Florida', 'Georgia_(U.S._state)',
       'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
       'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts',
       'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
       'Nevada', 'New_Hampshire', 'New_Jersey', 'New_Mexico', 'New_York',
       'North_Carolina', 'North_Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Rhode_Island', 'South_Carolina', 'South_Dakota',
       'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West_Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

To merge the two dataframes we need to rename some of the values in the State column for the scored cities. There are a couple cases where the formatting is strange. For example, North Carolina is formated as `North_Carolina` and Georgia is formatted as `Georgia_(U.S._state)`. To merge this with our regional data we need to correct the state column. This is done below.

In [19]:
# Replacing underscores with spaces and renaming the Georgia state
new_state_column = ["Georgia" if state == "Georgia_(U.S._state)" else state.replace('_', ' ') for state in us_cities_scored['state']]
# Updating the state column
us_cities_scored['state'] = new_state_column
# Remerging the dataframes
merged_df = us_cities_scored.merge(us_regions_reformat, how='outer', on='state')

After the merge there are only two states that appear to not have any articles associated with them, Connecticut and Nebraska.

In [129]:
merged_df[merged_df['page_title'].isnull()]

,state,page_title,revision_id,score,url,region,division
22157,Connecticut,NaN,<NA>,NaN,NaN,Northeast,New England
22158,Nebraska,NaN,<NA>,NaN,NaN,Midwest,West North Central


This means that from the US States by Region file we have two states, Connecticut and Nebraska that don't have any associated wikipedia page of a city associated with them. However, we will still keep them in our dataset since they are valid states.

### Step 2: Adding Census Data
The next step is to add Census data to our dataset. We first load in the dataset using pandas. Rename the columns, remove null rows and columns and reformat the state column

In [20]:
census_df = pd.read_excel("../data_raw/NST-EST2022-POP.xlsx", header=None, skiprows=9, nrows=53, usecols=[0, 3])
census_df.columns = ['state', 'population']

# Removing null rows and columns
census_df = census_df.dropna(how="all")

# Reformatting the state column
census_df['state'] = [state.replace('.', '') for state in census_df['state']]

# Changing population to int type
census_df['population'] = census_df['population'].astype(int)

# Merging
final_df = merged_df.merge(census_df, how="outer", on="state")

Here we can see there are two "states" that should be removed. The District of Columbia and Puerto Rico. These are not valid states. These are shown below. Note that we also see Connecticut and Nebraska because we have no articles of cities on these two states.

In [21]:
final_df[final_df['page_title'].isnull()]

,state,page_title,revision_id,score,url,region,division,population
22157,Connecticut,NaN,<NA>,NaN,NaN,Northeast,New England,3623355
22158,Nebraska,NaN,<NA>,NaN,NaN,Midwest,West North Central,1963554
22159,District of Columbia,NaN,<NA>,NaN,NaN,NaN,NaN,668791
22160,Puerto Rico,NaN,<NA>,NaN,NaN,NaN,NaN,3262693


In [22]:
# Removing the invalid states
invalid_states = ["District of Columbia", "Puerto Rico"]
invalid_mask = (final_df["state"] != invalid_states[0]) & (final_df["state"] != invalid_states[1])
final_df = final_df[invalid_mask]

The final steps to clean up our merged dataframe is to remove duplicates and rename the columns to the proper formatting.

In [23]:
# Removing duplicates
final_no_dup = final_df.drop_duplicates()

# Renaming columns for final formatting
final_no_dup.columns = ['state', 'article_title', 'revision_id', 'article_quality', 'url', 'region', 'regional_division', 'population']
final_subset = final_no_dup[['state', 'regional_division', 'population', 'article_title', 'revision_id', 'article_quality']]

# Saving final dataframe
final_subset.to_csv("../data_final/wp_scored_city_articles_by_state.csv", index=False)